<a href="https://colab.research.google.com/github/txusser/Master_IA_Sanidad/blob/main/Modulo_3/NLP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalamos algunas dependencias.

In [1]:
!mkdir tutorial
!git clone https://github.com/scikit-learn/scikit-learn.git
!pip install scikit-learn
!cp -r scikit-learn/doc/tutorial/text_analytics/ tutorial
!python tutorial/text_analytics/data/twenty_newsgroups/fetch_data.py

mkdir: cannot create directory ‘tutorial’: File exists
fatal: destination path 'scikit-learn' already exists and is not an empty directory.


Nos descargamos el dataset que vamos a utilizar en este ejemplo. 
Es un dataset de e-mails (aproximadamente 12.000 documentos en total), divididos (casi) uniformemente en 20 grupos de dependiendo de los temas que se tratan. Fue recopilado originalmente por Ken Lang, probablemente para su artículo "Newsweeder: Learning to filter netnews".

In [2]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train',shuffle=True)
twenty_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [3]:
len(twenty_train.data)

11314

In [4]:
for t in twenty_train.target[:10]:
  print(twenty_train.target_names[t])

rec.autos
comp.sys.mac.hardware
comp.sys.mac.hardware
comp.graphics
sci.space
talk.politics.guns
sci.med
comp.sys.ibm.pc.hardware
comp.os.ms-windows.misc
comp.sys.mac.hardware


In [5]:
print("\n".join(twenty_train.data[4].split("\n")))

From: jcm@head-cfa.harvard.edu (Jonathan McDowell)
Subject: Re: Shuttle Launch Question
Organization: Smithsonian Astrophysical Observatory, Cambridge, MA,  USA
Distribution: sci
Lines: 23

From article <C5owCB.n3p@world.std.com>, by tombaker@world.std.com (Tom A Baker):
>>In article <C5JLwx.4H9.1@cs.cmu.edu>, ETRAT@ttacs1.ttu.edu (Pack Rat) writes...
>>>"Clear caution & warning memory.  Verify no unexpected
>>>errors. ...".  I am wondering what an "expected error" might
>>>be.  Sorry if this is a really dumb question, but
> 
> Parity errors in memory or previously known conditions that were waivered.
>    "Yes that is an error, but we already knew about it"
> I'd be curious as to what the real meaning of the quote is.
> 
> tom


My understanding is that the 'expected errors' are basically
known bugs in the warning system software - things are checked
that don't have the right values in yet because they aren't
set till after launch, and suchlike. Rather than fix the code
and possibly i

‎El preprocesamiento de texto, la tokenización y el filtrado de palabras clave se incluyen en ‎‎CountVectorizer‎‎, que crea un diccionario de características y transforma los emails en vectores de características:.



In [6]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(11314, 130107)

Veamos el tamaño de nuestro vocabulario

In [7]:
count_vect.vocabulary_.get(u'algorithm')

27366

‎El recuento de ocurrencias es un buen comienzo, pero hay un problema: los documentos más largos tendrán valores de recuento promedio más altos que los documentos más cortos, aunque puedan hablar sobre los mismos temas.‎

‎Para evitar estas posibles discrepancias basta con dividir el número de apariciones de cada palabra en un documento por el número total de palabras en el documento: estas nuevas características se denominan Frecuencias de Término (tf).

‎Otro refinamiento además de transformar los valores en tf es reducir los pesos de las palabras que aparecen en muchos documentos del corpus y, por lo tanto, son menos informativas que las que ocurren solo en una porción más pequeña del corpus.‎

‎Esta reducción de escala se denomina ‎‎tf-idf‎‎ para "Term Frequency times Inverse Document Frequency".‎

‎Tanto ‎‎tf‎‎ como ‎‎tf-idf‎‎ se pueden calcular de la siguiente manera utilizando ‎‎TfidfTransformer‎‎:‎

In [8]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)

‎En el código de ejemplo anterior, en primer lugar usamos el método "*fit*" para ajustar nuestro estimador a los datos y, en segundo lugar, el método "*transform*" para transformar nuestra matriz de conteo en una representación tf-idf. Estos dos pasos se pueden combinar para lograr el mismo resultado final al omitir el procesamiento redundante. Esto se hace mediante el uso del método "*fit_transform*". Ejercicio: Reemplazar por el método "*fit_transform*"

In [9]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(11314, 130107)

‎Ahora que tenemos nuestras features, podemos entrenar un clasificador para que intente predecir la categoría de un post. Comencemos con un clasificador ‎‎naive Bayes, que proporciona una buena línea de base para esta tarea. Existen varias variantes de este clasificador; la más adecuada para el recuento de palabras es la variante multinomial

In [10]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

Para tratar de predecir el resultado en un nuevo documento, necesitamos extraer las características utilizando casi la misma cadena de extracción de características que antes. La diferencia es que llamamos a "*transform*" en lugar de a "*fit_transform*", ya que ya se han ajustado al conjunto de entrenamiento

In [11]:
docs_new = ['Jesus Loves You. Follow his guidance and you will be able to reach heaven', 'Earth is 7 light years away from the sun','You need to ensure that the engine temperature does not reach above 90 degrees']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
  print('%r => %s' % (doc, twenty_train.target_names[category]))

'Jesus Loves You. Follow his guidance and you will be able to reach heaven' => soc.religion.christian
'Earth is 7 light years away from the sun' => sci.space
'You need to ensure that the engine temperature does not reach above 90 degrees' => rec.autos


‎Para que el vectorizador = > transformador = > clasificador sea más fácil de trabajar, proporciona una clase ‎‎Pipeline‎‎ que se comporta como un clasificador compuesto

In [12]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', MultinomialNB())])
text_clf.fit(twenty_train.data, twenty_train.target)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

Podemos utilizar la misma base de datos para evaluar la precisión predictiva del modelo. Para eso, la base de datos reserva una porción test:‎



In [13]:
import numpy as np
twenty_test = fetch_20newsgroups(subset='test',shuffle=True)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.7738980350504514

‎Logramos una precisión de alrededor del 75%. Veamos si podemos hacerlo mejor con una support vector machine (SVM),‎‎ que es ampliamente considerado como uno de los mejores algoritmos de clasificación de texto (aunque también es un poco más lento que el naive Bayes):‎



In [14]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42,max_iter=5, tol=None))])
text_clf.fit(twenty_train.data, twenty_train.target)
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.8248805098247477

Hemos subido alrededor de un 5% en rendimiento utilizando una SVM. scikit-learn‎ proporciona más utilidades para un análisis de rendimiento más detallado de los resultados:‎

In [15]:
from sklearn import metrics
print(metrics.classification_report(twenty_test.target, predicted,target_names=twenty_test.target_names))


                          precision    recall  f1-score   support

             alt.atheism       0.73      0.71      0.72       319
           comp.graphics       0.78      0.72      0.75       389
 comp.os.ms-windows.misc       0.73      0.78      0.75       394
comp.sys.ibm.pc.hardware       0.74      0.67      0.70       392
   comp.sys.mac.hardware       0.81      0.83      0.82       385
          comp.windows.x       0.84      0.76      0.80       395
            misc.forsale       0.84      0.90      0.87       390
               rec.autos       0.91      0.90      0.90       396
         rec.motorcycles       0.93      0.96      0.95       398
      rec.sport.baseball       0.88      0.90      0.89       397
        rec.sport.hockey       0.88      0.99      0.93       399
               sci.crypt       0.84      0.96      0.90       396
         sci.electronics       0.83      0.62      0.71       393
                 sci.med       0.87      0.86      0.87       396
         

In [16]:
for i in twenty_train.target_names:
  for j in twenty_train.target_names:
    if i!=j and metrics.confusion_matrix(twenty_test.target, predicted)[twenty_train.target_names.index(i),twenty_train.target_names.index(j)]>20:
      print ('%s  -  %s = %s' % (i,j,metrics.confusion_matrix(twenty_test.target, predicted)[twenty_train.target_names.index(i),twenty_train.target_names.index(j)])) 

alt.atheism  -  soc.religion.christian = 44
comp.graphics  -  comp.os.ms-windows.misc = 21
comp.graphics  -  comp.windows.x = 24
comp.os.ms-windows.misc  -  comp.sys.ibm.pc.hardware = 21
comp.sys.ibm.pc.hardware  -  comp.os.ms-windows.misc = 27
comp.sys.ibm.pc.hardware  -  comp.sys.mac.hardware = 26
comp.sys.ibm.pc.hardware  -  sci.electronics = 22
comp.sys.mac.hardware  -  comp.sys.ibm.pc.hardware = 22
comp.windows.x  -  comp.graphics = 32
comp.windows.x  -  comp.os.ms-windows.misc = 42
rec.sport.baseball  -  rec.sport.hockey = 32
sci.electronics  -  comp.sys.ibm.pc.hardware = 22
sci.electronics  -  sci.crypt = 30
talk.politics.misc  -  talk.politics.guns = 102
talk.religion.misc  -  alt.atheism = 45
talk.religion.misc  -  soc.religion.christian = 56
talk.religion.misc  -  talk.politics.guns = 22


‎Ya hemos encontrado algunos parámetros como *use_idf* en el *TfidfTransformer*. Los clasificadores tienden a tener muchos parámetros; por ejemplo, *MultinomialNB* incluye un parámetro de suavizado *alpha* y *SGDClassifier* tiene un parámetro de penalización *alpha* y términos configurables de pérdida y penalización en la función objetivo.

‎Podemos intentar optimizar estos parámetros para mejorar nuestra clasificación. En lugar de ajustar los parámetros de los diversos componentes de la cadena, es posible ejecutar una búsqueda exhaustiva de los mejores parámetros en una cuadrícula de valores posibles. Probamos todos los clasificadores en palabras o bigrams, con o sin idf, y con un parámetro de penalización de 0.01 o 0.001 para el SVM lineal:

Obviamente, una búsqueda tan exhaustiva puede ser costosa. Si tenemos varios núcleos de CPU, podemos decirle al proceso que pruebe estas ocho combinaciones de parámetros en paralelo. Si le damos a este parámetro un valor de -1 a *n_jobs*, grid search detectará cuántos núcleos están instalados y los usará todos


In [20]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],'tfidf__use_idf': (True, False),'clf__alpha': (1e-2, 1e-3)}
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1)
gs_clf = gs_clf.fit(twenty_train.data[:1000], twenty_train.target[:1000])

‎Los objetos ‎best_score_ y best_params_ almacenan la mejor puntuación media y la configuración de parámetros correspondiente a esa puntuación:

In [21]:
gs_clf.best_score_

0.766